In [1]:
# def cv2_read_image_ensure_4_channels(img_path):
#     img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
#     if len(img.shape) == 2:  # Grayscale
#         img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
#     if img.shape[2] == 3:    # BGR
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
#     return img

In [2]:
import os
import cv2
import math
import numpy as np
import mediapipe as mp
os.makedirs("_output", exist_ok=True)

def expand_polygon(ls_keypoints, scale_factor):
    # Convert keypoints to a NumPy array
    points = np.array(ls_keypoints, dtype=np.float32)
    # Calculate the centroid of the polygon
    centroid = np.mean(points, axis=0)
    # Translate points to origin (subtract centroid), scale, and translate back
    expanded_points = centroid + (points - centroid) * scale_factor
    # Convert back to integer coordinates for OpenCV
    expanded_points = expanded_points.astype(np.int32)
    return expanded_points.tolist()

def get_keypoints(img_path, img_debug_path, expand_scale_factor):
    # Read image
    img_raw = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
    h, w, _ = img_raw.shape
    # MediaPipe: Extract face landmarks
    with mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
        results = face_mesh.process(img_rgb)
        if results.multi_face_landmarks:
            for facelandmarks in results.multi_face_landmarks:
                # Landmark indices
                ls_indices = [10,338,297,332,284,251,389,356,454,323,361,288,397,365,379,378,400,377,152,148,176,149,150,136,172,58,132,93,234,127,162,21,54,103,67,109]
                # Get coordinates
                ls_keypoints = []
                for idx in ls_indices:
                    ls_keypoints.append(
                        (int(facelandmarks.landmark[idx].x * w), int(facelandmarks.landmark[idx].y * h)) # A keypoint (x,y)
                    )
                # Expand polygon
                ls_keypoints_expanded = expand_polygon(ls_keypoints, expand_scale_factor)
                # Draw points
                cv2.polylines(img_raw, [np.array(ls_keypoints, dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=3)
                cv2.polylines(img_raw, [np.array(ls_keypoints_expanded, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=3)
                for keypoint in ls_keypoints:
                    cv2.circle(img_raw, keypoint, 4, (0, 255, 0), -1)
                cv2.imwrite(img_debug_path, img_raw)
                # Return
                return ls_keypoints_expanded, h, w
    return None # No faces detected

def create_mask(ls_keypoints, img_h, img_w, img_mask_path):
    # Convert to numpy array (required format for OpenCV fillPoly)
    ls_keypoints = np.array(ls_keypoints, np.int32)
    ls_keypoints = ls_keypoints.reshape((-1, 1, 2)) # shape (n,1,2)
    # Create a blank black image (size depends on your image)
    img_mask = np.zeros((img_h, img_w, 3), dtype=np.uint8)
    # Fill the polygon with white
    cv2.fillPoly(img_mask, [ls_keypoints], (255, 255, 255))
    cv2.imwrite(img_mask_path, img_mask)

In [3]:
img_path = "_test/img1.jpg"
ls_keypoints, img_h, img_w = get_keypoints(img_path, "_output/img_keypoints.jpg", expand_scale_factor=1.4)
create_mask(ls_keypoints, img_h, img_w, "_output/img_mask.jpg")

d:\GITHUB\IMAGEN\notebooks\head_replace\_venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
